# Imports

In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
import cv2
import matplotlib.pyplot as plt
import itertools
import torch_fidelity
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image
import torchvision.transforms as transforms
import pandas as pd
import random
import torch.nn.functional as F
import sys
from tqdm import tqdm

# SEED = 42
SEED = random.randrange(2**32 - 1)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
print("Random Seed:", SEED)


Random Seed: 1418598704


# Work

In [2]:
from models.generators import UNetGenerator, FastStyleGenerator
from models.discriminators import PatchDiscriminator, RandomKaggleDiscriminator, PatchSpectralDiscriminator

"""
Step 4. Initalize G and D¶
"""
G_AB = UNetGenerator()
D_B = PatchDiscriminator()
G_BA = UNetGenerator()
D_A = PatchDiscriminator()

## Total parameters in CycleGAN should be less than 60MB
total_params = sum(p.numel() for p in G_AB.parameters()) + \
               sum(p.numel() for p in G_BA.parameters()) + \
               sum(p.numel() for p in D_A.parameters()) + \
               sum(p.numel() for p in D_B.parameters())


total_params_million = total_params / (1024 * 1024)
print(f'Total parameters in CycleGAN model: {total_params_million:.2f} million')

Total parameters in CycleGAN model: 23.10 million


In [3]:
"""
Step 3. Define Loss
"""
from collections import defaultdict
from loss_functions import GradientPreservationLoss, IdentityPreservationLoss, PatchNCELoss, HingeAdversarialLoss, EdgeConsistencyLoss, LineContinuityLoss

# criterion_GAN = HingeAdversarialLoss()
criterion_GAN = nn.BCELoss()
criterion_cycle = nn.L1Loss()
criterion_identity = IdentityPreservationLoss()
criterion_gradient = GradientPreservationLoss()
criterion_contrast = PatchNCELoss()
# edge_loss = EdgeAwareLoss()


def dynamic_loss_weighting(epoch, total_epochs):
    # Early epochs: Focus on content preservation
    d = defaultdict(float)
    if epoch < total_epochs * 0.3:
        d.update({
            'identity': 5.0,
            'gan': 1.0,
            'cycle': 10.0,
            'facial_component': 2.0,
            'edge': 2.0,
        })
    elif epoch < total_epochs * 0.7:
        d.update({
            'identity': 2.0,
            'gan': 1.0,
            'cycle': 10.0,
            'line': 2.0,
            'facial_component': 5.0,
        })
    else:
        d.update({
            'identity': 1.0,
            'gan': 1.5,
            'cycle': 10.0,
            'line': 3.0,
            'facial_component': 5.0,
        })
    return d

In [4]:
if torch.cuda.is_available():
    print(f"Current GPU: {torch.cuda.current_device()}")
    print(f"Current GPU name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
    device = torch.device("cuda")
    G_AB = G_AB.cuda()
    D_B = D_B.cuda()
    G_BA = G_BA.cuda()
    D_A = D_A.cuda()
    criterion_GAN = criterion_GAN.cuda()
    criterion_cycle = criterion_cycle.cuda()
    criterion_identity = criterion_identity.cuda()
    Tensor = torch.cuda.FloatTensor
else:
    print("PyTorch does not have access to GPU, falling back to CPU")
    Tensor = torch.Tensor
    device = torch.device("cpu")



Current GPU: 0
Current GPU name: NVIDIA GeForce RTX 4070 Ti


In [5]:
"""
Step 5. Configure Optimizers
"""

def get_lr_scheduler(optimizer, n_epochs=100, n_epochs_decay=100, lr_policy='linear'):
    if lr_policy == 'linear':
        def lambda_rule(epoch):
            # Keep constant for first n_epochs, then linearly decay to zero
            lr_l = 1.0 - max(0, epoch - n_epochs) / float(n_epochs_decay + 1)
            return lr_l
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_rule)
    else:
        raise NotImplementedError(f'learning rate policy {lr_policy} not implemented')
    return scheduler

# Optimizer setup
optimizer_G = torch.optim.Adam(itertools.chain(G_AB.parameters(), G_BA.parameters()),
                               lr=0.0002, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(itertools.chain(D_A.parameters(), D_B.parameters()),
                               lr=0.0002, betas=(0.5, 0.999))

# Learning rate schedulers
scheduler_G = get_lr_scheduler(optimizer_G, n_epochs=100, n_epochs_decay=100)
scheduler_D = get_lr_scheduler(optimizer_D, n_epochs=100, n_epochs_decay=100)


In [6]:
from typing import Callable, Literal


def run_one_epoch(
    G_AB: nn.Module,
    G_BA: nn.Module,
    D_A: nn.Module,
    D_B: nn.Module,
    state: Literal["train", "eval"],
    dataloader: DataLoader,
    criterion_identity: Callable,
    criterion_GAN: Callable,
    criterion_cycle: Callable,
    optimizer_G: torch.optim.Optimizer,
    optimizer_D: torch.optim.Optimizer,
    # optimizer_D_A: torch.optim.Optimizer,
    # optimizer_D_B: torch.optim.Optimizer,
    lambdas: dict[str, float],
) -> dict[str, float]:

    # Set training/evaluation mode only when necessary
    if state == "train":
        G_AB.train(), G_BA.train()
        D_A.train(), D_B.train()
    else:
        G_AB.eval(), G_BA.eval()
        D_A.eval(), D_B.eval()

    running_losses = {
        "G": 0.0, "D_A": 0.0, "D_B": 0.0,
        "identity": 0.0, "gan": 0.0, "cycle": 0.0, "line": 0.0
    }

    # No gradient tracking during evaluation
    torch.set_grad_enabled(state == "train")
    with tqdm(dataloader, unit="batch", desc="Training" if state == "train" else "Validation") as tepoch:
        for real_A, real_B in tepoch:
            optimizer_G.zero_grad()
            real_A, real_B = real_A.to(device, dtype=torch.float32, non_blocking=True), real_B.to(device, dtype=torch.float32, non_blocking=True)

            # Prepare ground truth labels
            valid, fake = torch.ones(real_A.size(0), 1, device=device, requires_grad=False), torch.zeros(real_A.size(0), 1, device=device, requires_grad=False)

            # Train Generators
            fake_B = G_AB(real_A)
            fake_A = G_BA(real_B)

            # Identity loss
            loss_id_A = criterion_identity(fake_B, real_A)
            loss_id_B = criterion_identity(fake_A, real_B)
            loss_identity = (loss_id_A + loss_id_B) / 2

            # GAN loss
            loss_GAN_AB = criterion_GAN(D_B(fake_B), valid)
            loss_GAN_BA = criterion_GAN(D_A(fake_A), valid)
            loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2

            # Cycle loss
            recov_A = G_BA(fake_B)
            recov_B = G_AB(fake_A)
            loss_cycle_A = criterion_cycle(recov_A, real_A)
            loss_cycle_B = criterion_cycle(recov_B, real_B)
            loss_cycle = (loss_cycle_A + loss_cycle_B) / 2

            # # Line continuity loss
            # loss_line_A = line_loss(fake_A, real_A)
            # loss_line_B = line_loss(fake_B, real_B)
            # loss_line = (loss_line_A + loss_line_B) / 2

            # Total generator loss
            loss_G = lambdas['identity'] * loss_identity + lambdas['gan'] * loss_GAN + lambdas['cycle'] * loss_cycle

            if state == "train":
                loss_G.backward()
                optimizer_G.step()

                optimizer_D.zero_grad()
                # Train Discriminators
                # optimizer_D_A.zero_grad()
                # optimizer_D_B.zero_grad()
                
                loss_real_A = criterion_GAN(D_A(real_A), valid)
                loss_fake_A = criterion_GAN(D_A(fake_A.detach()), fake)
                loss_D_A = (loss_real_A + loss_fake_A) / 2
                
                loss_real_B = criterion_GAN(D_B(real_B), valid)
                loss_fake_B = criterion_GAN(D_B(fake_B.detach()), fake)
                loss_D_B = (loss_real_B + loss_fake_B) / 2
                
                loss_D = (loss_D_A + loss_D_B)
                loss_D.backward()
                optimizer_D.step()
                # optimizer_D_A.step()
                # optimizer_D_B.step()

            # Accumulate losses
            running_losses["G"] += loss_G.item()
            running_losses["D_A"] += loss_D_A.item()
            running_losses["D_B"] += loss_D_B.item()
            running_losses["identity"] += loss_identity.item()
            running_losses["gan"] += loss_GAN.item()
            running_losses["cycle"] += loss_cycle.item()
            # running_losses["line"] += loss_line.item()

    print(f'[G loss: {loss_G.item()} | identity: {loss_identity.item()} GAN: {loss_GAN.item()} cycle: {loss_cycle.item()}]')
    print(f'[D loss: {loss_D.item()} | D_A: {loss_D_A.item()} D_B: {loss_D_B.item()}]')
    # Average the losses over the dataset
    return {k: v / len(dataloader) for k, v in running_losses.items()}

In [7]:
"""
Step 6. DataLoader
"""
from CustomImageDataset import CustomImageDataset as ImageDataset
# data_dir = '/kaggle/input/group-project/image_image_translation'
data_dir = ''

image_size = (128, 128)
transforms_ = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

loader_params = {
    "batch_size": 20,
    "num_workers": 2,
    "pin_memory": False,
    "shuffle": True,
    # "prefetch_factor": 2,
    "persistent_workers": True
}


trainloader = DataLoader(
    ImageDataset(data_dir, mode='train', transform=transforms_),
    **loader_params
)

loader_params["shuffle"] = False
validloader = DataLoader(
    ImageDataset(data_dir, mode='valid', transform=transforms_),
    **loader_params
)

In [8]:
"""
Step 7. Training
"""
n_epochs = 200
for epoch in range(1, n_epochs + 1):
    print(f'[Epoch {epoch}/{n_epochs}]')
    run_one_epoch(
        G_AB, G_BA, D_A, D_B, "train", trainloader,
        criterion_identity, criterion_GAN, criterion_cycle,
        optimizer_G, optimizer_D,
        dynamic_loss_weighting(epoch, n_epochs)
    )

    # validation
    if epoch % 25 == 0:
    #     valid_real_A, valid_real_B = next(iter(testloader))
    #     sample_images(valid_real_A, valid_real_B)

    #     loss_D = (loss_D_A + loss_D_B) / 2
    #     print(f'[Epoch {epoch+1}/{n_epochs}]')
    #     print(f'[G loss: {loss_G.item()} | identity: {loss_identity.item()} GAN: {loss_GAN.item()} cycle: {loss_cycle.item()}]')
    #     print(f'[D loss: {loss_D.item()} | D_A: {loss_D_A.item()} D_B: {loss_D_B.item()}]')


        # Save model checkpoints  
        torch.save(G_AB.state_dict(), f'checkpoints/G_AB_{epoch}.pth')
        torch.save(D_A.state_dict(), f'checkpoints/D_A_{epoch}.pth')
        torch.save(G_BA.state_dict(), f'checkpoints/G_BA_{epoch}.pth')
        torch.save(D_B.state_dict(), f'checkpoints/D_B_{epoch}.pth')


[Epoch 1/200]


Training: 100%|██████████| 160/160 [01:20<00:00,  1.99batch/s]


[G loss: 3.1249022483825684 | identity: 0.11807626485824585 GAN: 0.7539727687835693 cycle: 0.1780548244714737]
[D loss: 1.298529863357544 | D_A: 0.6687344908714294 D_B: 0.6297953128814697]
[Epoch 2/200]


Training: 100%|██████████| 160/160 [01:14<00:00,  2.13batch/s]


[G loss: 3.084479331970215 | identity: 0.09351526945829391 GAN: 0.8887666463851929 cycle: 0.17281362414360046]
[D loss: 1.09140944480896 | D_A: 0.5438265800476074 D_B: 0.5475828647613525]
[Epoch 3/200]


...

[G loss: 1.7292804718017578 | identity: 0.18626132607460022 GAN: 0.717125654220581 cycle: 0.0467330738902092]
[D loss: 1.302215814590454 | D_A: 0.6584782600402832 D_B: 0.6437375545501709]
[Epoch 199/200]


Training: 100%|██████████| 160/160 [01:11<00:00,  2.23batch/s]


[G loss: 1.7903443574905396 | identity: 0.18468546867370605 GAN: 0.747728705406189 cycle: 0.048406586050987244]
[D loss: 1.2710974216461182 | D_A: 0.6333482265472412 D_B: 0.637749195098877]
[Epoch 200/200]


Training: 100%|██████████| 160/160 [01:11<00:00,  2.23batch/s]

[G loss: 1.8236892223358154 | identity: 0.17696642875671387 GAN: 0.8128668069839478 cycle: 0.04274224862456322]
[D loss: 1.271430253982544 | D_A: 0.6459071636199951 D_B: 0.625523030757904]


# Output Results

In [9]:
from typing import Type
def evaluate(
    model: torch.nn.Module,
    input_dir: str,
    output_dir: str,
    ref_dir: str,
    batch_size: int,
    image_size: int,
    tensor_type: Type[torch.Tensor]
) -> float:
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    generate_transforms = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    to_image = transforms.ToPILImage()

    files = [os.path.join(input_dir, name) for name in os.listdir(input_dir)]

    model.eval()
    for i in range(0, len(files), batch_size):
        # Read and transform images
        imgs = [generate_transforms(Image.open(files[j])) for j in range(i, min(len(files), i + batch_size))]
        imgs = torch.stack(imgs, 0).type(tensor_type)

        # Generate images
        fake_imgs = model(imgs).detach().cpu()

        # Save generated images
        for j in range(fake_imgs.size(0)):
            img = fake_imgs[j].squeeze().permute(1, 2, 0).numpy()
            img = (img - np.min(img)) * 255 / (np.max(img) - np.min(img))
            img = to_image(img.astype(np.uint8))
            _, name = os.path.split(files[i + j])
            img.save(os.path.join(output_dir, name))

    # Compute metrics
    metrics: dict[str, float] = torch_fidelity.calculate_metrics(
        input1=output_dir,
        input2=ref_dir,
        cuda=True,
        fid=True,
        isc=True
    )

    fid_score: float = metrics["frechet_inception_distance"]
    is_score: float = metrics["inception_score_mean"]

    if is_score > 0:
        gms: float = np.sqrt(fid_score / is_score)
        print("Geometric Mean Score:", gms)
        return gms, fid_score, is_score
    else:
        print("IS is 0, GMS cannot be computed!")
        return 0, 0, 0


In [10]:
# Parameters
image_size = 128
batch_size = loader_params["batch_size"]

# data_dir = '/kaggle/input/group-project/image_image_translation'
data_dir = '.'


def format_score(score):
    return "{:.4f}".format(score).replace(".", "_")

def report_score(s_value_1, fid_1, is_1, s_value_2, fid_2, is_2, epoch=None):
    s_value = np.round((s_value_1+s_value_2)/2, 5)
    df = pd.DataFrame({'id': [1], 'label': [s_value]})
    filename = f"{format_score(s_value)}-C[{format_score(s_value_1)}-{format_score(fid_1)}-{format_score(is_1)}]-R[{format_score(s_value_2)}-{format_score(fid_2)}-{format_score(is_2)}]"
    if epoch: filename = f"{epoch}-{filename}"
    csv_path = filename+".csv"
    df.to_csv(csv_path, index=False)

    print(f"CSV saved to {csv_path}")

for i in range(25, 201, 25):
    G_AB.load_state_dict(torch.load(f'checkpoints/G_AB_{i}.pth'))
    G_BA.load_state_dict(torch.load(f'checkpoints/G_BA_{i}.pth'))

    # Raw to Cartoon
    s_value_1, fid_1, is_1 = evaluate(
        model=G_AB,
        input_dir=os.path.join(data_dir, 'VAE_generation/test'),
        output_dir='../Cartoon_images',
        ref_dir=f"{data_dir}/VAE_generation_Cartoon/test",
        batch_size=batch_size,
        image_size=image_size,
        tensor_type=Tensor
    )

    # Cartoon to Raw
    s_value_2, fid_2, is_2 = evaluate(
        model=G_BA,
        input_dir=os.path.join(data_dir, 'VAE_generation_Cartoon/test'),
        output_dir='../Raw_images',
        ref_dir=f"{data_dir}/VAE_generation/test",
        batch_size=batch_size,
        image_size=image_size,
        tensor_type=Tensor
    )

    report_score(s_value_1, fid_1, is_1, s_value_2, fid_2, is_2, epoch=i)



Creating feature extractor "inception-v3-compat" with features ['logits_unbiased', '2048']
Extracting features from input1
Looking for samples non-recursivelty in "../Cartoon_images" with extensions png,jpg,jpeg
Found 1000 samples, some are lossy-compressed - this may affect metrics
c:\Users\a\Desktop\cs424-project\venv313\Lib\site-packages\torch_fidelity\datasets.py:16: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(img.tobytes())).view(height, width, 3)
Processing samples                                                          
Extracting features from input2
Looking for samples non-recursivelty in "./VAE_generation_Cartoon/test" with extensions png,jpg,jpeg
Found 1000 samples, some are lossy-co

Geometric Mean Score: 6.5877892521854955


Creating feature extractor "inception-v3-compat" with features ['logits_unbiased', '2048']
Extracting features from input1
Looking for samples non-recursivelty in "../Raw_images" with extensions png,jpg,jpeg
Found 1000 samples, some are lossy-compressed - this may affect metrics
Processing samples                                                          
Extracting features from input2
Looking for samples non-recursivelty in "./VAE_generation/test" with extensions png,jpg,jpeg
Found 1000 samples, some are lossy-compressed - this may affect metrics
Processing samples                                                          
Inception Score: 3.4337070849858513 ± 0.3293053879799815
Frechet Inception Distance: 81.76578219756959


Geometric Mean Score: 4.879824190118329
CSV saved to 25-5_7338-C[6_5878-158_0364-3_6415]-R[4_8798-81_7658-3_4337].csv


Creating feature extractor "inception-v3-compat" with features ['logits_unbiased', '2048']
Extracting features from input1
Looking for samples non-recursivelty in "../Cartoon_images" with extensions png,jpg,jpeg
Found 1000 samples, some are lossy-compressed - this may affect metrics
Processing samples                                                           
Extracting features from input2
Looking for samples non-recursivelty in "./VAE_generation_Cartoon/test" with extensions png,jpg,jpeg
Found 1000 samples, some are lossy-compressed - this may affect metrics
Processing samples                                                          
Inception Score: 3.467344978075018 ± 0.2413272173684885
Frechet Inception Distance: 143.36902941322595


...

Creating feature extractor "inception-v3-compat" with features ['logits_unbiased', '2048']
Extracting features from input1
Looking for samples non-recursivelty in "../Cartoon_images" with extensions png,jpg,jpeg
Found 1000 samples, some are lossy-compressed - this may affect metrics
Processing samples                                                          
Extracting features from input2
Looking for samples non-recursivelty in "./VAE_generation_Cartoon/test" with extensions png,jpg,jpeg
Found 1000 samples, some are lossy-compressed - this may affect metrics
Processing samples                                                          
Inception Score: 2.949787488936179 ± 0.24784572571636135
Frechet Inception Distance: 87.1563050461802


Geometric Mean Score: 5.435681986238997


Creating feature extractor "inception-v3-compat" with features ['logits_unbiased', '2048']
Extracting features from input1
Looking for samples non-recursivelty in "../Raw_images" with extensions png,jpg,jpeg
Found 1000 samples, some are lossy-compressed - this may affect metrics
Processing samples                                                          
Extracting features from input2
Looking for samples non-recursivelty in "./VAE_generation/test" with extensions png,jpg,jpeg
Found 1000 samples, some are lossy-compressed - this may affect metrics
Processing samples                                                          
Inception Score: 3.636556642548969 ± 0.2531730643329214


Geometric Mean Score: 3.6179389151299817
CSV saved to 200-4_5268-C[5_4357-87_1563-2_9498]-R[3_6179-47_6006-3_6366].csv


Frechet Inception Distance: 47.60064269139451


# Clean Up

In [11]:
import gc
del G_AB, G_BA, D_A, D_B

gc.collect()
with torch.no_grad():
    torch.cuda.empty_cache()